In [25]:
from itertools import combinations
import numpy as np
from icecream import ic
from collections import namedtuple
import random
import matplotlib.pyplot as plt
import os

## Common tests

In [26]:
def common_tests(problem):
    negatives = np.any(problem < 0)
    zero_diagonal = np.all(np.diag(problem) == 0)
    symmetric = np.all(problem == problem.T)

    print("Are there negative values?", negatives)
    print("Are all diagonal values zero?", zero_diagonal)
    print("Is the matrix symmetric?", symmetric)
    
    return negatives, zero_diagonal, symmetric


## Genetic algorithm utilities

In [27]:
Individual = namedtuple('Individual', ['genotype', 'fitness'])

In [28]:
def fitness(genotype, num_cities, problem):
    total_cost = 0
    for i in range(num_cities):
        total_cost += problem[genotype[i], genotype[(i + 1) % num_cities]]
    return total_cost

### Parent selection

In [29]:
class ParentSelector:
    """Collection of selection methods for evolutionary algorithms."""

    def tournament(self, population, tau=3) -> Individual:
        tournament = random.sample(population, tau)
        return min(tournament, key=lambda ind: ind.fitness)

    def rank(self, population) -> Individual:
        sorted_population = sorted(population, key=lambda ind: ind.fitness)
        ranks = list(range(len(sorted_population), 0, -1))
        total_rank = sum(ranks)
        probabilities = [rank / total_rank for rank in ranks]
        return random.choices(sorted_population, weights=probabilities)[0]



### Genetic operators

In [30]:
class GeneticOperator:
    """Collection of mutation / crossover operators."""

    def __init__(self, num_cities):
        self.num_cities = int(num_cities)

    # ---------- Mutations ----------
    def swap_mutation(self, individual: Individual) -> Individual:
        idx1 = random.randrange(self.num_cities)
        idx2 = random.randrange(self.num_cities)
        new_genotype = individual.genotype[:]
        new_genotype[idx1], new_genotype[idx2] = new_genotype[idx2], new_genotype[idx1]
        return Individual(new_genotype, None)

    def insertion_mutation(self, individual: Individual) -> Individual:
        idx1 = random.randrange(self.num_cities)
        idx2 = random.randrange(self.num_cities)
        city = individual.genotype[idx1]
        new_genotype = individual.genotype[:idx1] + individual.genotype[idx1 + 1:]
        new_genotype = new_genotype[:idx2] + [city] + new_genotype[idx2:]
        return Individual(new_genotype, None)

    def inversion_mutation(self, individual: Individual) -> Individual:
        idx1 = random.randint(0, self.num_cities - 2)
        idx2 = random.randint(idx1 + 1, self.num_cities - 1)
        new_genotype = individual.genotype[:]
        new_genotype[idx1:idx2 + 1] = list(reversed(new_genotype[idx1:idx2 + 1]))
        return Individual(new_genotype, None)

    
    # ---------- Crossovers ----------

    def ordered_crossover(self, parent1: Individual, parent2: Individual) -> Individual:
        idx1 = random.randint(0, self.num_cities - 2)
        idx2 = random.randint(idx1 + 1, self.num_cities - 1)
        child = [None] * self.num_cities
        child[idx1:idx2 + 1] = parent1.genotype[idx1:idx2 + 1]
        cur = (idx2 + 1) % self.num_cities
        for city in parent2.genotype:
            if city not in child:
                child[cur] = city
                cur = (cur + 1) % self.num_cities
        return Individual(child, None)
    
    def order_based_crossover(self, parent1: Individual, parent2: Individual) -> Individual:
        positions = sorted(random.sample(range(self.num_cities), k=random.randint(1, self.num_cities - 1)))
        selected_cities = [parent2.genotype[i] for i in positions]
        child = parent1.genotype[:]
        # Remove selected cities from child
        for city in selected_cities:
            child.remove(city)
        # Reinsert selected cities in the same order as in parent2 at the original positions
        for pos, city in zip(positions, selected_cities):
            child.insert(pos, city)
        return Individual(child, None)


    def position_based_crossover(self, parent1: Individual, parent2: Individual) -> Individual:
        idx1 = random.randint(0, self.num_cities - 2)
        idx2 = random.randint(idx1 + 1, self.num_cities - 1)
        positions = list(range(idx1, idx2 + 1))
        selected = [parent1.genotype[i] for i in positions]
        child = [None] * self.num_cities
        for pos in positions:
            child[pos] = parent1.genotype[pos]
        cur = 0
        for city in parent2.genotype:
            if city not in selected:
                while child[cur] is not None:
                    cur += 1
                child[cur] = city
        return Individual(child, None)


    # ---------- Edge recombination helpers ----------
    def create_edge_map(self, parent1: Individual, parent2: Individual) -> dict:
        edge_map = {city: set() for city in range(self.num_cities)}
        t1 = parent1.genotype
        t2 = parent2.genotype
        for city in range(self.num_cities):
            i1 = t1.index(city)
            i2 = t2.index(city)
            edge_map[city].add(t1[(i1 - 1) % self.num_cities])
            edge_map[city].add(t1[(i1 + 1) % self.num_cities])
            edge_map[city].add(t2[(i2 - 1) % self.num_cities])
            edge_map[city].add(t2[(i2 + 1) % self.num_cities])
        return edge_map

    def genetic_edge_recombination(self, parent1: Individual, parent2: Individual) -> Individual:
        edge_map = self.create_edge_map(parent1, parent2)
        initial = random.choice(list(parent1.genotype))
        child = [initial]
        visited = {initial}
        current = initial
        while len(child) < self.num_cities:
            neighbors = edge_map.get(current, set()).copy()
            for nb in neighbors:
                edge_map[nb].discard(current)
            edge_map.pop(current, None)
            if neighbors:
                min_count = min(len(edge_map.get(n, set())) for n in neighbors)
                candidates = [n for n in neighbors if len(edge_map.get(n, set())) == min_count]
                nxt = random.choice(candidates)
            else:
                unvisited = [c for c in range(self.num_cities) if c not in visited]
                if not unvisited:
                    break
                nxt = random.choice(unvisited)
            child.append(nxt)
            visited.add(nxt)
            current = nxt
        return Individual(child, None)
    
    def inver_crossover(self, parent1: Individual, parent2: Individual):
        idx1 = random.randint(0, self.num_cities - 1)
        gene = parent1.genotype[idx1]

        # find position of gene in parent2 and its successor there
        idx2_in_p2 = parent2.genotype.index(gene)
        next_city = parent2.genotype[(idx2_in_p2 + 1) % self.num_cities]

        idx_next_in_p1 = parent1.genotype.index(next_city)

        # if positions coincide nothing to do
        if idx1 == idx_next_in_p1:
            child_genotype = parent1.genotype[:]
        elif idx1 < idx_next_in_p1:
            # invert the segment (idx1+1 .. idx_next_in_p1) inclusive at the right end
            segment = list(reversed(parent1.genotype[idx1 + 1: idx_next_in_p1 + 1]))
            child_genotype = parent1.genotype[:idx1 + 1] + segment + parent1.genotype[idx_next_in_p1 + 1:]
        else:
            # idx_next_in_p1 < idx1: invert the wrap-around segment (idx_next_in_p1 .. idx1-1)
            segment = list(reversed(parent1.genotype[idx_next_in_p1: idx1]))
            child_genotype = parent1.genotype[:idx_next_in_p1] + segment + parent1.genotype[idx1:]

        return Individual(child_genotype, None)
    
    def cycle_crossover(self, parent1: Individual, parent2: Individual) -> Individual:
        child_genotype = [None] * self.num_cities
        indices = set(range(self.num_cities))
        while indices:
            start = indices.pop()
            idx = start
            while True:
                child_genotype[idx] = parent1.genotype[idx]
                next_city = parent2.genotype[idx]
                idx = parent1.genotype.index(next_city)
                if idx == start:
                    break
                indices.discard(idx)
        for i in range(self.num_cities):
            if child_genotype[i] is None:
                child_genotype[i] = parent2.genotype[i]
        return Individual(child_genotype, None)

### Initial population

In [31]:
def nearest_neighbor(problem, num_cities, k_range=(1, 3)):
    start = random.randrange(num_cities)
    unvisited = set(range(num_cities))
    tour = [start]
    unvisited.remove(start)
    current = start

    while unvisited:
        k = random.randint(k_range[0], k_range[1])
        sorted_neighbors = sorted(unvisited, key=lambda c: problem[current][c])
        candidates = sorted_neighbors[:min(k, len(sorted_neighbors))]
        next_city = random.choice(candidates)
        tour.append(next_city)
        unvisited.remove(next_city)
        current = next_city
    return tour

In [32]:
def ga_algorithm(problem, population, num_steps, offspring_size, mutation_rate, population_size, mutation_fn, crossover_fn, parent_selection_fn):
    fitness_history = []

    for _ in range(num_steps):
        offspring = []
        for _ in range(offspring_size):
            if random.random() < mutation_rate:
                p = parent_selection_fn(population)
                o = mutation_fn(p)
            else:
                p1 = parent_selection_fn(population)
                p2 = parent_selection_fn(population)
                o = crossover_fn(p1, p2)
            offspring.append(Individual(o.genotype, fitness(o.genotype, num_cities=problem.shape[0], problem=problem)))

        population.extend(offspring)
        population = sorted(population, key=lambda ind: ind.fitness)[:population_size]
        fitness_history.append(population[0].fitness)

    return fitness_history


In [33]:
directory = "tests"
for file in os.listdir(directory):
    print(file)
    filepath = os.path.join(directory, file)
    problem = np.load(filepath)

    negatives, zero_diagonal, symmetric = common_tests(problem)
    
    num_cities = problem.shape[0]   
    population_size = 50
    offspring_size = 20
    mutation_rate = 0.5
    num_steps = 10*num_cities
    
    operators = GeneticOperator(num_cities)
    parent_selector = ParentSelector()

    operators = GeneticOperator(num_cities)
    parent_selector = ParentSelector()

    mutation_methods = {
        "swap": operators.swap_mutation,
        "insertion": operators.insertion_mutation,
        "inversion": operators.inversion_mutation,
    }

    crossover_methods = {  
        "ox1": operators.ordered_crossover,
        "pos": operators.position_based_crossover,
        "edge": operators.genetic_edge_recombination,
        "ox2": operators.order_based_crossover,
        "inv": operators.inver_crossover,
        "cx": operators.cycle_crossover,
    }

    parent_selections = {
        "tournament": parent_selector.tournament,
        "rank": parent_selector.rank
    }

    # if g problem: inversion+edge+rank
    # if r1 problem: inversion+ox2+tournament
    # if r2 problem: invversion+ox1+tournament
                
    if "g" in file:
        m_name, m_fn = "inversion", operators.inversion_mutation
        c_name, c_fn = "edge", operators.genetic_edge_recombination
        p_name, p_fn = "rank", parent_selector.rank
    elif "r1" in file:
        m_name, m_fn = "inversion", operators.inversion_mutation
        c_name, c_fn = "ox2", operators.order_based_crossover
        p_name, p_fn = "tournament", parent_selector.tournament
    elif "r2" in file:
        m_name, m_fn = "inversion", operators.inversion_mutation
        c_name, c_fn = "ox1", operators.ordered_crossover
        p_name, p_fn = "tournament", parent_selector.tournament
        
    current_population = []
    for _ in range(population_size):
        tour = nearest_neighbor(problem, num_cities)
        cost = fitness(tour, num_cities, problem)
        current_population.append(Individual(tour,cost))
    scores = ga_algorithm(problem,current_population, num_steps, offspring_size, mutation_rate, population_size, m_fn, c_fn, p_fn)
    print(f"{file} - {m_name} + {c_name} + {p_name}: Best Fitness = {scores[-1]}")
    #plt.plot(scores)
    #plt.show()
            

problem_g_10.npy
Are there negative values? False
Are all diagonal values zero? True
Is the matrix symmetric? True
problem_g_10.npy - inversion + edge + rank: Best Fitness = 1497.6636482252907
problem_g_100.npy
Are there negative values? False
Are all diagonal values zero? True
Is the matrix symmetric? True
problem_g_100.npy - inversion + edge + rank: Best Fitness = 4262.414876851067
problem_g_1000.npy
Are there negative values? False
Are all diagonal values zero? True
Is the matrix symmetric? True
problem_g_1000.npy - inversion + edge + rank: Best Fitness = 16108.03234174387
problem_g_20.npy
Are there negative values? False
Are all diagonal values zero? True
Is the matrix symmetric? True
problem_g_20.npy - inversion + edge + rank: Best Fitness = 1755.5146770830045
problem_g_200.npy
Are there negative values? False
Are all diagonal values zero? True
Is the matrix symmetric? True
problem_g_200.npy - inversion + edge + rank: Best Fitness = 6270.6062846054165
problem_g_50.npy
Are there ne